# ReadME - Background

The nonprofit foundation Alphabet Soup wants a tool that can help it select the applicants for funding with the best chance of success in their ventures. With your knowledge of machine learning and neural networks, you’ll use the features in the provided dataset to create a binary classifier that can predict whether applicants will be successful if funded by Alphabet Soup.

From Alphabet Soup’s business team, you have received a CSV containing more than 34,000 organizations that have received funding from Alphabet Soup over the years. Within this dataset are a number of columns that capture metadata about each organization, such as:

* EIN and NAME—Identification columns
* APPLICATION_TYPE—Alphabet Soup application type
* AFFILIATION—Affiliated sector of industry
* CLASSIFICATION—Government organization classification
* USE_CASE—Use case for funding
* ORGANIZATION—Organization type
* STATUS—Active status
* INCOME_AMT—Income classification
* SPECIAL_CONSIDERATIONS—Special considerations for application
* ASK_AMT—Funding amount requested
* IS_SUCCESSFUL—Was the money used effectively



# Step 1: Preprocess the Data

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique(axis=0)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9", "T13", "T12", "T2", "T25", "T14", "T29", "T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
vc = application_df["CLASSIFICATION"].value_counts()
vc[vc>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ["C7000", "C1700", "C4000", "C5000", "C1270", "C2700",
                              "C2800", "C7100", "C1300", "C1280", "C1230", "C1400",
                              "C7200", "C2300", "C1240", "C8000", "C7120", "C1500",
                              "C1800", "C6000", "C1250", "C8200", "C1238", "C1278",
                              "C1235", "C1237", "C7210", "C2400", "C1720", "C4100",
                              "C1257", "C1600", "C1260", "C2710", "C0", "C3200",
                              "C1234", "C1246", "C1267", "C1256", "C1370", "C1236",
                              "C2190", "C5200", "C1580", "C2570", "C4200", "C6100",
                              "C2600", "C2380", "C1900", "C1820", "C1248", "C3700",
                              "C2500", "C1570", "C1283", "C2170", "C1732", "C2561",
                              "C1245", "C4500", "C8210", "C4120", "C1728", "C2150"]
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(application_df)
dummies

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features and target arrays
y = dummies['IS_SUCCESSFUL']
X = dummies.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Step 2: Compile, Train, and Evaluate the Model

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputs = len(X_train_scaled[0])

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=inputs))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 220       
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.6410 - accuracy: 0.6800
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5706 - accuracy: 0.7256
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5639 - accuracy: 0.7265
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5610 - accuracy: 0.7284
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5594 - accuracy: 0.7294
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5583 - accuracy: 0.7293
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5575 - accuracy: 0.7298
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5572 - accuracy: 0.7292
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5566 - accuracy: 0.7297
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5564 - accuracy: 0.7294

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5555 - accuracy: 0.7306 - 935ms/epoch - 3ms/step
Loss: 0.5554879307746887, Accuracy: 0.7306122183799744


In [42]:
# Export our model to HDF5 file
filepath = r"C:\Users\Morgen\Deep_Learning_Challenge\deep-learning-challenge\Model.h5"
nn_model.save(filepath, save_format='h5')

# Step 3: Optimize the Model

In [51]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_2 = tf.keras.models.Sequential()

# First hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=inputs))

# Second hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn_model_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 10)                440       
                                                                 
 dense_11 (Dense)            (None, 10)                110       
                                                                 
 dense_12 (Dense)            (None, 1)                 11        
                                                                 
Total params: 561
Trainable params: 561
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Compile the model
nn_model_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [53]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8822 - accuracy: 0.5348 - 372ms/epoch - 1ms/step
Loss: 0.8822139501571655, Accuracy: 0.534810483455658


In [54]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_3 = tf.keras.models.Sequential()

# First hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=inputs))

# Second hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Third hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer
nn_model_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 10)                440       
                                                                 
 dense_14 (Dense)            (None, 10)                110       
                                                                 
 dense_15 (Dense)            (None, 15)                165       
                                                                 
 dense_16 (Dense)            (None, 1)                 16        
                                                                 
Total params: 731
Trainable params: 731
Non-trainable params: 0
_________________________________________________________________


In [55]:
# Compile the model
nn_model_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [63]:
# Train the model
fit_model_3 = nn_model_3.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 1ms/step - loss: 0.5997 - accuracy: 0.6884
Epoch 2/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5603 - accuracy: 0.7267
Epoch 3/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5557 - accuracy: 0.7275
Epoch 4/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7276
Epoch 5/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5528 - accuracy: 0.7299
Epoch 6/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5513 - accuracy: 0.7303
Epoch 7/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5506 - accuracy: 0.7296
Epoch 8/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5498 - accuracy: 0.7309
Epoch 9/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5499 - accuracy: 0.7299
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accura

In [64]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5572 - accuracy: 0.7305 - 244ms/epoch - 909us/step
Loss: 0.5571819543838501, Accuracy: 0.7304956316947937


# Step 4: Report on the Neural Network Model

**Overview**

This anaylsis will help the nonprofit foundation Alphabet Soup decide which of their funding ventures are most likely to be fruitful.

**Results**

* *Data Preprocessing*

  * **What variable(s) are the target(s) for your model?**
  The y variable, or the "IS_SUCCESSFUL" column is the target for my model.
  * **What variable(s) are the features for your model?**
  The X variable, or all columns except for the "IS_SUCCESSFUL" column are the features for my model.
  * **What variable(s) should be removed from the input data because they are neither targets nor features?**
  The "EIN" and "NAME" columns were removed because they were neither targets nor features.


* *Compiling, Training, and Evaluating the Model*
  * **How many neurons, layers, and activation functions did you select for your neural network model, and why?**
  I would chose the model with two hidden layers with 5 and 10 neurons respectively. This would be my choice because this was my most accurate model.
  * **Were you able to achieve the target model performance?**
  No, though I was only 2% away from this goal.
  * **What steps did you take in your attempts to increase model performance?**
  Adding more layers, neurons and epochs. I thought this would give the model more ability to process the information.



**Summary**

The results of my most accurate model were as follows:



In [62]:
# Make a prediction using the testing data
predictions = nn_model.predict(X_test)
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
print("Confusion Matrix")
display(cm_df)

268/268 [==============================] - 1s 3ms/step
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,0,4010
Actual 1,0,4565


This means that my model is assuming that every venture will be successful. I think that a model with fewer layer and neurons will be more successful because the models that are less complicated have functioned more accurately.

# ReadME - References

IRS. Tax Exempt Organization Search Bulk Data Downloads. https://www.irs.gov/